## **Hugging Face Token**

Cette cellule nous permet de récupérer le token d'API Hugging Face pour l'authentification.

In [2]:
import os
from dotenv import load_dotenv
from transformers import AutoTokenizer

# Chargement du fichier .env
load_dotenv()

# Récupération du token
access_token = os.getenv("HUGGING_FACE_TOKEN")
model_id = "FacebookAI/roberta-base"

if access_token:
    tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token)
    print(f" Tokenizer chargé avec succès")
else:
    print("Erreur : HUGGING_FACE_TOKEN introuvable")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

 Tokenizer chargé avec succès


___

## **Définition des labels**

Ici on définit les catégories d'arguments que le modèle doit identifer en utilisant le format BIO (Begin, Inside, Outside).

In [7]:
# O : Hors argument, B- : Début d'un segment, I- : Intérieur d'un segment
label_list = [
    "O",
    "B-MajorClaim", "I-MajorClaim",
    "B-Claim", "I-Claim",
    "B-Premise", "I-Premise"
]

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

print(f"Labels configurés : {label2id}")

Labels configurés : {'O': 0, 'B-MajorClaim': 1, 'I-MajorClaim': 2, 'B-Claim': 3, 'I-Claim': 4, 'B-Premise': 5, 'I-Premise': 6}


___

## **Chargement et Extraction des séquences**
On a besoin d'extracter les listes de mots (tokens) ainsi que leur étiquettes d'argumentation correspondantes. Donc on charge les fichiers JSON depuis le dossier `data`

In [ ]:
import json
from pathlib import Path

data_dir = Path("data")

def load_and_extract(file_name):
    """
    Charge un fichier JSON et extrait les séquences de mots et leurs étiquettes BIO.
    
    Args:
        file_name (str): Nom du fichier à charger dans le dossier data.
        
    Returns:
        tuple: (sentences, labels) où chaque élément est une liste de listes.
    """
    path = data_dir / file_name
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    sentences = []
    labels = []

    for doc in data:
        # Parcours des paragraphes pour extraire BIO tags
        for paragraph in doc["tokens"]:
            if not paragraph: 
                continue

            words = [token["str"] for token in paragraph]
            tags = [token["arg"] for token in paragraph]

            sentences.append(words)
            labels.append(tags)

    return sentences, labels

train_texts, train_tags = load_and_extract("aae_train.json")
dev_texts, dev_tags = load_and_extract("aae_dev.json")

print(f"{len(train_texts)} phrases chargées pour l'entraînement.")
print(f"Exemple de tokens : {train_texts[0][:5]}")
print(f"Exemple de tags   : {train_tags[0][:5]}")

1592 phrases chargées pour l'entraînement.
Exemple de tokens : ['Should', 'students', 'be', 'taught', 'to']
Exemple de tags   : ['O', 'O', 'O', 'O', 'O']


__________
## **Préparation et Alignements des Données**

Cette cellule prépare les données pour le modèle en alignant les étiquettes BIO avec les jetons (tokens) générés par RoBERTa. Elle gère notamment le découpage des mots en sous-unités et ignore les jetons spéciaux (CLS, SEP) via l'index `-100`. Les données sont ensuite encapsulées dans un objet `Dataset` PyTorch pour l'entraînement.

In [ ]:
import torch

def tokenize_and_align_labels(texts, tags):
    """
    Tokenize les textes et aligne les étiquettes BIO avec les sous-unités de mots (sub-tokens).
    Gère les jetons spéciaux avec l'index -100 pour les ignorer lors du calcul de la perte.
    """
    tokenized_inputs = tokenizer(
        texts,
        truncation=True,
        is_split_into_words=True,
        padding='max_length',
        max_length=512
    )

    labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            # Index -100 pour ignorer les jetons spéciaux (CLS, SEP, Padding)
            if word_idx is None:
                label_ids.append(-100)
            # Assignation du label au premier sub-token du mot ou aux suivants
            elif word_idx != previous_word_idx:
                label_ids.append(label2id.get(label[word_idx], 0))
            else:
                label_ids.append(label2id.get(label[word_idx], 0))
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_tokenized = tokenize_and_align_labels(train_texts, train_tags)
dev_tokenized = tokenize_and_align_labels(dev_texts, dev_tags)

class ArgumentDataset(torch.utils.data.Dataset):
    """
    Encapsulation des données tokenisées dans un objet Dataset PyTorch.
    Convertit les listes de listes en tenseurs PyTorch au moment de l'accès.
    """
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Conversion dynamique de chaque clé (input_ids, attention_mask, labels)
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = ArgumentDataset(train_tokenized)
dev_dataset = ArgumentDataset(dev_tokenized)

___
## **Initialisation du modèle RoBERTA**
Cette étape charge le modèle pré-entraîné RoBERTa et le configure pour notre tâche de classification (Argument Mining). On définit la correspondance entre les étiquettes (ID/Label) et on transfère le modèle sur le **GPU (CUDA)** pour accélérer l'entraînement.

In [10]:
import torch
from transformers import AutoModelForTokenClassification

# On définit RoBERTa-base avec nos labels spécifiques
model = AutoModelForTokenClassification.from_pretrained(
    model_id,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    token=access_token
)

# Détection automatique de la disponibilité du GPU (Cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

print(f"Modèle sur : {device}")

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForTokenClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
classifier.weight               | MISSING    | 
classifier.bias                 | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Modèle sur : cuda


___
## **Configuration et Entrainement (Essais)**
On définit ici les hyperparamètres (vitesse d'apprentissage, taille des lots, nombre d'époques) et on initialise le `Trainer` de Hugging Face. Cette cellule lance l'entraînement spécifique sur le jeu de données des **Essais** et sauvegarde le meilleur modèle à la fin du processus.

In [ ]:
from transformers import TrainingArguments, Trainer

# Configuration (optimisée pour GPU < 12GB)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
)

from transformers import Trainer, DataCollatorForTokenClassification

# padding
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
)

print("Entrainement en cours...")
trainer.train()
trainer.save_model("./best_arg_model")

Entrainement en cours...


Epoch,Training Loss,Validation Loss
1,0.806944,0.743488
2,0.575058,0.544287
3,0.480258,0.532692


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

___
## **Evaluation de la généralisation**

Cette étape teste la capacité du modèle, entraîné uniquement sur les **essais**, à prédire des arguments dans le domaine **médical**. En comparant la perte (loss) entre le domaine source et les domaines cibles, on visualise directement **le conflit de domaine** (domain shift) : le modèle peine à généraliser ses connaissances à cause de la spécificité du jargon médical.

In [ ]:
import pandas as pd

test_files = {
    "Essays (AAE - Source)": "aae_test.json",
    "Med. Neoplasm (Cible)": "abstrct_neoplasm_test.json",
    "Med. Glaucoma (Cible)": "abstrct_glaucoma_test.json",
    "Med. Mixed (Cible)": "abstrct_mixed_test.json"
}

all_metrics = []

print(f"{'Domaine de test':<25} | {'Perte (Loss)':<15}")
print("-" * 45)

# Boucle d'évaluation
for domain_name, file_path in test_files.items():
    try:
        
        texts, tags = load_and_extract(file_path)
        encodings = tokenize_and_align_labels(texts, tags)
        dataset = ArgumentDataset(encodings)

        output = trainer.predict(dataset)
        loss = output.metrics.get("test_loss", 0)

        all_metrics.append({
            "Domaine": domain_name,
            "Loss": round(loss, 4),
            "Echantillons": len(texts)
        })

        print(f"{domain_name:<25} | {loss:<15.4f}")

    except Exception as e:
        print(f" Erreur sur {domain_name}: {e}")

df_results = pd.DataFrame(all_metrics)
print("\n--- Tableau Récapitulatif ---")
print(df_results)

Domaine de test           | Perte (Loss)   
---------------------------------------------


Essays (AAE - Source)     | 0.4506         


Med. Neoplasm (Cible)     | 2.3469         


Med. Glaucoma (Cible)     | 1.4923         


Med. Mixed (Cible)        | 2.2833         

--- Tableau Récapitulatif ---
                 Domaine    Loss  Echantillons
0  Essays (AAE - Source)  0.4506           444
1  Med. Neoplasm (Cible)  2.3469            24
2  Med. Glaucoma (Cible)  1.4923           224
3     Med. Mixed (Cible)  2.2833            20


On observe une perte (**Loss**) cinq fois plus élevée sur les données médicales que sur les essais. Ce résultat confirme que le modèle ne parvient pas à généraliser ses connaissances à cause de la spécificité du vocabulaire technique et scientifique. Le transfert de domaine direct est ici un échec, rendant les prédictions médicales peu fiables en l'état.

Les résultats précédents montrent que le modèle entraîné sur les essais ne peut pas être transféré tel quel au domaine médical sans une perte majeure de précision. Pour remédier à cet échec de généralisation, nous divisons la suite du projet en deux étapes : une **Partie 1** dédiée à la consolidation du modèle sur son domaine d'origine (Essais) et une **Partie 2 **consacrée à un ré-entraînement spécifique sur le corpus médical pour s'adapter à sa complexité technique.

___
## **Partie 1 : Apprentissage sur le domain des Essais**
Cette étape est dédiée à l'apprentissage du modèle sur le jeu de données **AAE** (`aae_*.json`), spécialisé dans les structures argumentatives d'essais étudiants. Le processus englobe le chargement des données, l'initialisation de **RoBERTa** et un fine-tuning sur 6 époques. L'objectif est de créer un modèle de référence capable de capturer avec précision les segments de texte (spans) et leurs rôles logiques dans un langage courant.

In [ ]:

train_texts, train_tags = load_and_extract("aae_train.json")
dev_texts, dev_tags = load_and_extract("aae_test.json")


train_encodings = tokenize_and_align_labels(train_texts, train_tags)
dev_encodings = tokenize_and_align_labels(dev_texts, dev_tags)

train_dataset = ArgumentDataset(train_encodings)
dev_dataset = ArgumentDataset(dev_encodings)

# Init modèle RoBERTA
model_essay = AutoModelForTokenClassification.from_pretrained(
    model_id,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    token=access_token
).to(device)

# Config Train
training_args_essay = TrainingArguments(
    output_dir="./results_essays",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
)

# Trainer
trainer_essay = Trainer(
    model=model_essay,
    args=training_args_essay,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
)

print("Entraînement en cours...")
trainer_essay.train()
trainer_essay.save_model("./model_essays_final")
print("Modèle sauvegardé.")

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForTokenClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
classifier.weight               | MISSING    | 
classifier.bias                 | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Entraînement en cours...


Epoch,Training Loss,Validation Loss
1,0.958172,1.251725
2,0.678323,0.536729
3,0.568704,0.505746
4,0.471017,0.521043
5,0.402858,0.453977
6,0.346909,0.466922


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Modèle sauvegardé.


___
## **Partie 2 : Apprentissage sur le domaine Médical**

Cette étape adapte le modèle aux spécificités du discours scientifique en utilisant les corpus médicaux **Glaucoma** et **Mixed** (`abstrct_*.json`) pour l'entraînement. En combinant ces données techniques, le pipeline permet à **RoBERTa** d'ajuster ses poids au vocabulaire médical complexe. Le jeu de données **Neoplasm** est utilisé comme référence de validation pour mesurer la capacité du modèle à identifier les structures argumentatives au sein de publications spécialisées.

In [19]:
# Combine Glaucoma et Mixed pour le train
texts_glaucoma, tags_glaucoma = load_and_extract("abstrct_glaucoma_test.json")
texts_mixed, tags_mixed = load_and_extract("abstrct_mixed_test.json")

med_train_texts = texts_glaucoma + texts_mixed
med_train_tags = tags_glaucoma + tags_mixed

med_dev_texts, med_dev_tags = load_and_extract("abstrct_neoplasm_test.json")

med_train_encodings = tokenize_and_align_labels(med_train_texts, med_train_tags)
med_dev_encodings = tokenize_and_align_labels(med_dev_texts, med_dev_tags)

med_train_dataset = ArgumentDataset(med_train_encodings)
med_dev_dataset = ArgumentDataset(med_dev_encodings)

# Init nouveau modèle
model_med = AutoModelForTokenClassification.from_pretrained(
    model_id,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    token=access_token
).to(device)

# Config Train
training_args_med = TrainingArguments(
    output_dir="./results_medical",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
)

# Trainer
trainer_med = Trainer(
    model=model_med,
    args=training_args_med,
    train_dataset=med_train_dataset,
    eval_dataset=med_dev_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
)

print("Entraînement en cours...")
trainer_med.train()
trainer_med.save_model("./model_medical_final")
print("Modèle sauvegardé.")

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForTokenClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
classifier.weight               | MISSING    | 
classifier.bias                 | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Entraînement en cours...


Epoch,Training Loss,Validation Loss
1,0.972113,0.878228
2,0.714119,0.680590
3,0.495754,0.597080
4,0.272895,0.513155
5,0.224144,0.483026
6,0.225239,0.473143


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Modèle sauvegardé.


___
##**Exportation des prédictions et formatage final**

Cette cellule génère les fichiers JSON finaux contenant les prédictions des modèles pour les deux domaines. Le processus inclut un nettoyage crucial des données : la suppression des clés spans originales pour forcer le script d'évaluation à utiliser uniquement les nouveaux labels BIO prédits par nos modèles. Ce formatage garantit une évaluation juste et rigoureuse de la capacité du modèle à délimiter les arguments par lui-même.

In [ ]:
import json
import numpy as np
import os

def final_export(file_name, current_trainer, output_name):
    """
    Génère un fichier JSON de prédictions compatible avec le script d'évaluation.
    Récupère les labels prédits par le modèle et les réinjecte dans la structure originale.
    """
    
    path_to_file = f"data/{file_name}" if os.path.exists(f"data/{file_name}") else file_name
    with open(path_to_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    texts, tags = load_and_extract(file_name)
    encodings = tokenize_and_align_labels(texts, tags)
    dataset = ArgumentDataset(encodings)

    output = current_trainer.predict(dataset)
    predictions = np.argmax(output.predictions, axis=-1)

    pred_idx = 0
    for doc in data:
        # Nettoyage pour l'évaluation (script evaluation.py)
        if 'spans' in doc:
            del doc['spans']

        for paragraph in doc["tokens"]:
            if not paragraph:
                continue

            current_preds = predictions[pred_idx]
            for i, token in enumerate(paragraph):
                if i < len(current_preds):
                    token["arg"] = id2label[current_preds[i]]
            pred_idx += 1

    with open(output_name, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Fichier '{output_name}' généré.")


# ESSAIS
final_export("aae_test.json", trainer_essay, "predictions_essays.json")

# MÉDICAL
final_export("abstrct_neoplasm_test.json", trainer_med, "predictions_neoplasm_final.json")
final_export("abstrct_glaucoma_test.json", trainer_med, "predictions_glaucoma_final.json")

Fichier 'predictions_essays.json' généré.


Fichier 'predictions_neoplasm_final.json' généré.


Fichier 'predictions_glaucoma_final.json' généré.


___
## **Evaluation finale et Rapport de Performance**
Cette étape ultime combine deux méthodes de mesure pour valider la qualité de nos modèles. D'une part, le script d'évaluation externe fournit une analyse des segments (spans) selon des critères stricts et assouplis.

D'autre part, la bibliothèque `seqeval` génère un rapport de classification détaillé, calculant la précision, le rappel et le score F1 pour chaque type de label (Claim, Premise). Cette double approche permet de confirmer la précision chirurgicale du modèle sur les essais et son adaptation réussie au domaine médical.

In [40]:
def print_header(title):
    print("\n" + "="*50)
    print(f"{title}")
    print("="*50)

print_header("ÉVALUATION : ESSAIS (AAE)")
!python evaluate.py data/aae_test.json predictions_essays.json

print_header("ÉVALUATION : MÉDECINE (NEOPLASM)")
!python evaluate.py data/abstrct_neoplasm_test.json predictions_neoplasm_final.json



ÉVALUATION : ESSAIS (AAE)


********************** SPANS *************************** 
   STRICT EVALUATION
    > Argument mining spans (unlabeled)
      Precision : 0.0016421693636883509 
      Recall    : 0.0019528870236241794
      F-score   : 0.0017781796262808921
    > Argument mining spans (labeled)
      Precision : 0.0011900897976847343 
      Recall    : 0.001319975631219116 
      F-score   : 0.0012507534659433395

    RELAXED EVALUATION (α = 0.5)
    > Argument mining spans (unlabeled)
      Precision : 0.4144769044398122 
      Recall    : 0.8069113512558078
      F-score   : 0.5467650196111312
    > Argument mining spans (labeled)
      Precision : 0.3591953441189353 
      Recall    : 0.6451334584824023 
      F-score   : 0.4603479825711268



******************* RELATIONS *************************** 
   STRICT EVALUATION
    > Argument mining spans (unlabeled)
      Precision : 1.0 
      Recall    : 1.0
      F-score   : 1.0
    > Argument mining spans (labeled)
      P

Avec le fichier abstrct.glaucoma.test.json, nous observons que le modèle échoue à identifier les segments argumentatifs sur ce domaine en retournant l'erreur `division by zero`. Plus précisément, aucune étiquette de début de segment `B-` n'est prédite. Cette absence totale de détection empêche le calcul des métriques de performance, car le modèle classifie l'intégralité du texte technique comme "Hors argument" `O`. Cela confirme que les marqueurs linguistiques du glaucome sont invisibles pour un modèle entraîné uniquement sur des essais.

In [ ]:
#!pip install seqeval

from seqeval.metrics import classification_report

def evaluate_and_report(current_trainer, current_dataset, name):
    """
    Calcule et affiche un rapport de classification détaillé (Precision, Recall, F1)
    en utilisant la bibliothèque seqeval, adaptée aux séquences BIO.
    """

    output = current_trainer.predict(current_dataset)
    preds = np.argmax(output.predictions, axis=-1)
    labels = output.label_ids

    # Nettoyage des étiquettes 
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]

    print(f"\n--- RAPPORT D'ÉVALUATION DÉTAILLÉ : {name} ---")
    print(classification_report(true_labels, true_predictions))


# ESSAIS
evaluate_and_report(trainer_essay, dev_dataset, "ESSAIS (AAE)")

# MÉDICAL
evaluate_and_report(trainer_med, med_dev_dataset, "MÉDECINE (ABSTRCT)")


 RAPPORT D'ÉVALUATION : ESSAIS (AAE)
              precision    recall  f1-score   support

       Claim       0.28      0.45      0.34       376
  MajorClaim       0.30      0.55      0.39       183
     Premise       0.58      0.68      0.63       922

   micro avg       0.44      0.60      0.51      1481
   macro avg       0.39      0.56      0.45      1481
weighted avg       0.47      0.60      0.52      1481




 RAPPORT D'ÉVALUATION : MÉDECINE (ABSTRCT)
              precision    recall  f1-score   support

       Claim       0.05      0.08      0.06        52
  MajorClaim       0.00      0.00      0.00         6
     Premise       0.30      0.41      0.35       147

   micro avg       0.23      0.32      0.27       205
   macro avg       0.12      0.16      0.14       205
weighted avg       0.23      0.32      0.27       205



/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
